In [1]:
!pip install -q --upgrade \
  transformers \
  datasets \
  peft==0.8.2 \
  accelerate \
  bitsandbytes \
  trl==0.8.6 \
  wandb \
  evaluate \
  rouge_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.8 MB/s eta 0:00:00

In [3]:
from huggingface_hub import login
login()  # Then paste your updated access token

In [2]:
!nvidia-smi

Thu May 22 21:49:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

Load Dataset

In [4]:
from datasets import load_dataset

# Load the full dataset (52K examples, subset later)
dolly = load_dataset("databricks/databricks-dolly-15k")

# Preview it
dolly["train"][0]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

{'instruction': 'When did Virgin Australia start operating?',
 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.",
 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.',
 'category': 'closed_qa'}

In [5]:
small_dolly = dolly["train"].shuffle(seed=42).select(range(1000))  # take 1k for training


In [11]:
import pandas as pd
pd.DataFrame(small_dolly.select(range(5)))

,instruction,context,response,category
0,Who were the children of the legendary Garth G...,,"Garth the Gardener, John the Oak, Gilbert of t...",open_qa
1,Give me a list of basic ingredients for baking...,,"To bake cookies you will need flour, sugar, eg...",brainstorming
2,Write a funny and whimsical horoscope reading,,"The stars say you should be patient, and that ...",creative_writing
3,Who was the first person to do spacewalk?,,"Alexei Leonov, a soviet cosmonaut was the firs...",open_qa
4,What causes an airplane wing to fly?,,"As Bernoulli's principal explains, because of ...",open_qa


In [31]:
def format_dolly(example):
    instruction = example['instruction']
    context = example.get('context', '')
    response = example['response']

    prompt = f"### Instruction:\n{instruction}\n\n"
    if context.strip():
        prompt += f"### Context:\n{context}\n\n"
    prompt += f"### Response:\n{response}"

    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()  # 👈 this is the key!
    return tokenized



In [32]:
tokenized_dataset = small_dolly.map(format_dolly)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [40]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "bigscience/bloomz-1b1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # needed for padding

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,179,648 || all params: 1,066,493,952 || trainable%: 0.11060990995662018


In [41]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./bloomz-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


<ipython-input-41-02f9e8152878>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,10.306700
20,7.997300
30,4.353800
40,4.270900
50,3.309800
60,3.418900
70,3.216200
80,2.857300
90,2.746700
100,3.697200


TrainOutput(global_step=500, training_loss=2.9919137229919435, metrics={'train_runtime': 220.5513, 'train_samples_per_second': 9.068, 'train_steps_per_second': 2.267, 'total_flos': 4184938119168000.0, 'train_loss': 2.9919137229919435, 'epoch': 2.0})

SAVE LOARA

In [42]:
model.save_pretrained("bloomz-1b1-lora")


Perform basic inference after fine-tuning to generate outputs.

add trained lora to base model

In [43]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load tokenizer and base model
model_id = "bigscience/bloomz-1b1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Load base + LoRA adapter
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, "./bloomz-1b1-lora")
model.eval()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 1536)
        (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1536, out_features=4608, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4608, bias=False)
                )
               

In [44]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [54]:
test_dolly = dolly["train"].shuffle(seed=42).select(range(1000, 2000))



In [55]:
predictions, references = [], []

for ex in test_dolly:
    prompt = f"### Instruction:\n{ex['instruction']}\n\n### Context:\n{ex['context']}\n\n### Response:"
    output = generate_response(prompt)
    predictions.append(output.split("### Response:")[-1].strip())
    references.append(ex["response"])



In [59]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics import accuracy_score

# BLEU
bleu_scores = [sentence_bleu([ref.split()], pred.split()) for ref, pred in zip(references, predictions)]

# ROUGE-L
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_scores = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, predictions)]

# Exact Match (optional)
exact_matches = [ref.strip().lower() == pred.strip().lower() for ref, pred in zip(references, predictions)]



In [60]:
print("Average BLEU:", sum(bleu_scores) / len(bleu_scores))
print("Average ROUGE-L:", sum(rouge_l_scores) / len(rouge_l_scores))
print("Exact Match Accuracy:", sum(exact_matches) / len(exact_matches))



Average BLEU: 0.012448823118067808
Average ROUGE-L: 0.14564769624894439
Exact Match Accuracy: 0.007


In [61]:
predictions

['Kōsei Arima',
 'Paris-Brest',
 'When the weather is warm and dry, stones can be used in your landscape to create a natural look and feel.',
 'definition',
 '1959',
 'Answer not in context',
 'The Garshasp-nama is an epic poem by Asadi Tusi (died 1072/73). It has been described as one of the best epic poems in Persian literature, comparable to Shahnameh, by Ferdowsi, and the most important work of Asadi Tusi. Asadi Tusi completed the poem in 1066 and dedicated it to a certain Abu Dulaf, the ruler of Nakhjavan (nothing is known about him). The poem has also been',
 '1992',
 'The hybrid is a type of vehicle that can be used to transport people and goods.',
 'computer science',
 '#include <bits/stdc++.h>\nusing namespace std;\nint fib(int n, int m) {\n  int res = 1;\n  while (n) {\n    if (m & 1) res = res * fib(n, m - 1);\n    n = n * fib(n - 1, m - 1);\n  }\n  return res;\n}\nint main() {\n  int n, m;\n  cin >> n >> m;\n  cout << fib(',
 'to make a good choice of career to make a good 